# CIT Data Pipeline: Formatting

In this notebook, we ingest the avaiable, pre-populated data and format it for proper SQL uploading. 

In [4]:
import pandas as pd
import numpy as np

import requests

In [5]:
def conform_headers(to_conform: [], existing_header: []) -> []:
    conformed_header = []
    """Given the two lists representing column headers, make one 
    header conform to another.
    
    :param to_conform: Column to conform.
    :type to_conform: [str]
    
    :param existing_header: Column for existing conformation.
    :type existing_header: [str] 
    
    :return conformed_header: List of column header conforming to existing header
    :return type: [str]
    """
    for column in existing_header:
        if column in to_conform:
            conformed_header.append(column)
    
    return conformed_header

In [6]:
# Unit Test
def test_conform_headers():
    to_conform = ["yes", "no"]
    existing_header = ["no"]
    conformed_header = conform_headers(to_conform, existing_header)
    assert conformed_header == ["no"]    
test_conform_headers()

In [7]:
def cherry_pick_dataframe(dataframe, conformed_header) -> pd.DataFrame:
    picked_dataframe = pd.DataFrame(columns=conformed_header)
    for column in conformed_header:
        picked_dataframe[column] = dataframe[column]
    return picked_dataframe

## Load the Data

In [8]:
files = pd.read_excel("CIT_Newly_added_Catalog_0521.xlsx")
print("Current Header:")
print(list(files.columns))
files.head()

Current Header:
['Plan Name', 'Date Added', 'Suggested By', 'Url', 'Plan Resolution', 'Planning Method', 'Land Conservation ', 'Unnamed: 7', 'Unnamed: 8', 'RESTORE GOALS', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14']


,Plan Name,Date Added,Suggested By,Url,Plan Resolution,Planning Method,Land Conservation,Unnamed: 7,Unnamed: 8,RESTORE GOALS,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaT,NaN,NaN,NaN,NaN,Aquisition,Easement,Stewardship,Habitat,Water Quality,Resources/Species,Community Resilience,Gulf Economy,Code
1,Habitat Management Plan - Baldwin County Meado...,2017-12-11,Jeniffer Roberts,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,THE MOBILE PENINSULA - CORRIDOR MASTER PLAN,2017-12-11,Jeniffer Roberts,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Management Plan for the - Audubon Bird Sanctuary,2017-12-11,Jeniffer Roberts,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apalachee Region Comprehensive Economic Develo...,2018-02-27,FL Fish and Wildlife Conservation Commission,http://www.nado.org/wp-content/uploads/2014/08...,Regional,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN


This pings which file links actually point to PDFs. Written by Ethan.

## Conform Incoming Column Names with Existing Column Names

### Find Current Labels

### Definitions:
Existing labels  
**incoming_header:** header in the incoming CSV  
**existing_header:** header in the existing db   
**files_df_remapped:** relabel incoming header to match with existing column names. Extraneous columns are identified, but not removed.  
**files_df_reduced:** files_df_remapped but selecting columns which intersect with existing_header.  
**files_df_expanded:** files_df_reduced expanded to include all columns in existing_header, not just those intersecting with the previous state.  

**:**   

In [9]:
meshpoint = 6
excel_header_1 = list(files.columns)[0:meshpoint]
excel_header_2 = list(files.iloc[0,meshpoint:])
incoming_header = excel_header_1 + excel_header_2
print("Incoming Header: \n", incoming_header)

Incoming Header: 
 ['Plan Name', 'Date Added', 'Suggested By', 'Url', 'Plan Resolution', 'Planning Method', 'Aquisition', 'Easement ', 'Stewardship', 'Habitat', 'Water Quality ', 'Resources/Species', 'Community Resilience', 'Gulf Economy', 'Code']


In [10]:
existing_header = ['id', 'plan_name', 'plan_url', 
                   'plan_resolution', 'planning_method', 'aquisition', 
                   'easement', 'stewardship', 'plan_timeframe', 
                   'agency_lead', 'geo_extent', 'habit', 
                   'water_quality', 'resource_species', 'community_resilience', 
                   'ecosystem_resilience', 'gulf_economy', 'related_state',
                   'status', 'is_new', 'existing_planid', 'username']

print("Existing Header: \n", existing_header)

Existing Header: 
 ['id', 'plan_name', 'plan_url', 'plan_resolution', 'planning_method', 'aquisition', 'easement', 'stewardship', 'plan_timeframe', 'agency_lead', 'geo_extent', 'habit', 'water_quality', 'resource_species', 'community_resilience', 'ecosystem_resilience', 'gulf_economy', 'related_state', 'status', 'is_new', 'existing_planid', 'username']


## Strip Column Names

In [11]:
files.columns = incoming_header
files = files[1:]
files.head()

,Plan Name,Date Added,Suggested By,Url,Plan Resolution,Planning Method,Aquisition,Easement,Stewardship,Habitat,Water Quality,Resources/Species,Community Resilience,Gulf Economy,Code
1,Habitat Management Plan - Baldwin County Meado...,2017-12-11,Jeniffer Roberts,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,THE MOBILE PENINSULA - CORRIDOR MASTER PLAN,2017-12-11,Jeniffer Roberts,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Management Plan for the - Audubon Bird Sanctuary,2017-12-11,Jeniffer Roberts,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apalachee Region Comprehensive Economic Develo...,2018-02-27,FL Fish and Wildlife Conservation Commission,http://www.nado.org/wp-content/uploads/2014/08...,Regional,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN
5,Fishery Management Plan for Spanish Mackerel,2018-02-27,FL Fish and Wildlife Conservation Commission,http://sedarweb.org/docs/wsupp/S17RD03%20ASMFC...,GCR,NaN,NaN,NaN,NaN,NaN,NaN,Manage Spanish mackerel resourse,NaN,Minimize disruptions of markets for Spanish ma...,NaN


### Relabel The Incoming Column Names

In [12]:
# Reduced given dataframe. 
# Columns ending with '_1' do not exist in the current database header.
incoming_header_remap = ['plan_name', 'date_added_1', 'suggested_by_1', 
                         'plan_url', 'plan_resolution', 'planning_method', 
                         'aquisition', 'easement', 'stewardship', 
                         'habitat', 'water_quality', 'resource_species', 
                         'community_resilience', 'gulf_economy', 'code_1']

rename_dictionary = {}
for i in range(len(incoming_header)):
    rename_dictionary[incoming_header_remap[i]] = incoming_header[i]

# files.rename(columns=rename_dictionary).head()

### Remap The Given DataFrame To Conform To Existing Columns

In [13]:
files_df_remapped = pd.DataFrame(columns=incoming_header_remap)
# print(list(files_df_remapped.columns))

for column in incoming_header_remap:
    files_df_remapped[column] = files[rename_dictionary[column]]
    
files_df_remapped.head(2)    

,plan_name,date_added_1,suggested_by_1,plan_url,plan_resolution,planning_method,aquisition,easement,stewardship,habitat,water_quality,resource_species,community_resilience,gulf_economy,code_1
1,Habitat Management Plan - Baldwin County Meado...,2017-12-11,Jeniffer Roberts,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,THE MOBILE PENINSULA - CORRIDOR MASTER PLAN,2017-12-11,Jeniffer Roberts,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Check The Incoming Column Remapping Worked

In [14]:
print(files_df_remapped.columns, "\n")
print(files.columns, "\n")
column_number_difference = len(files_df_remapped.columns) - len(files.columns)
assert column_number_difference == 0
print("Column Number Difference:", column_number_difference)

Index(['plan_name', 'date_added_1', 'suggested_by_1', 'plan_url',
       'plan_resolution', 'planning_method', 'aquisition', 'easement',
       'stewardship', 'habitat', 'water_quality', 'resource_species',
       'community_resilience', 'gulf_economy', 'code_1'],
      dtype='object') 

Index(['Plan Name', 'Date Added', 'Suggested By', 'Url', 'Plan Resolution',
       'Planning Method', 'Aquisition', 'Easement ', 'Stewardship', 'Habitat',
       'Water Quality ', 'Resources/Species', 'Community Resilience',
       'Gulf Economy', 'Code'],
      dtype='object') 

Column Number Difference: 0


## Conform Incoming Labels With Existing Ones 

In [15]:
conformed_header = conform_headers(list(files_df_remapped.columns), existing_header)

In [16]:
files_df_reduced = cherry_pick_dataframe(files_df_remapped, conformed_header)
files_df_reduced.head(2)

,plan_name,plan_url,plan_resolution,planning_method,aquisition,easement,stewardship,water_quality,resource_species,community_resilience,gulf_economy
1,Habitat Management Plan - Baldwin County Meado...,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,THE MOBILE PENINSULA - CORRIDOR MASTER PLAN,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Expand Columns For Incoming Dataframe With New Columns

Source: https://towardsdatascience.com/4-methods-for-adding-columns-to-pandas-dataframes-dd0696863c16

In [19]:
files_df_expanded = files_df_reduced.copy()
existing_header_copy = existing_header

for i in range(len(existing_header_copy)):  
    current_column = existing_header[i] 
    if current_column not in  list(files_df_reduced.columns):
        files_df_expanded.insert(i, current_column, "null")

files_df_expanded.head(3)

,id,plan_name,plan_url,plan_resolution,planning_method,aquisition,easement,stewardship,plan_timeframe,agency_lead,...,water_quality,resource_species,community_resilience,ecosystem_resilience,gulf_economy,related_state,status,is_new,existing_planid,username
1,null,Habitat Management Plan - Baldwin County Meado...,na,NaN,NaN,NaN,NaN,NaN,null,null,...,NaN,NaN,NaN,null,NaN,null,null,null,null,null
2,null,THE MOBILE PENINSULA - CORRIDOR MASTER PLAN,na,NaN,NaN,NaN,NaN,NaN,null,null,...,NaN,NaN,NaN,null,NaN,null,null,null,null,null
3,null,Management Plan for the - Audubon Bird Sanctuary,na,NaN,NaN,NaN,NaN,NaN,null,null,...,NaN,NaN,NaN,null,NaN,null,null,null,null,null


## Post-processing Application

#### Enumerate ID

In [21]:
current_index = 344
print("len", len(files_df_expanded))
id_col = range(current_index, len(files_df_expanded) + current_index) 
files_df_expanded["id"] = id_col 

len 295


range(344, 639)

In [29]:
# `is_new` should be boolean
files_df_expanded["is_new"] = "false"

files_df_expanded["existing_planid"] = files_df_expanded["id"]

files_df_expanded.head(3)

,id,plan_name,plan_url,plan_resolution,planning_method,aquisition,easement,stewardship,plan_timeframe,agency_lead,...,water_quality,resource_species,community_resilience,ecosystem_resilience,gulf_economy,related_state,status,is_new,existing_planid,username
1,344,Habitat Management Plan - Baldwin County Meado...,na,NaN,NaN,NaN,NaN,NaN,null,null,...,NaN,NaN,NaN,null,NaN,null,null,false,344,null
2,345,THE MOBILE PENINSULA - CORRIDOR MASTER PLAN,na,NaN,NaN,NaN,NaN,NaN,null,null,...,NaN,NaN,NaN,null,NaN,null,null,false,345,null
3,346,Management Plan for the - Audubon Bird Sanctuary,na,NaN,NaN,NaN,NaN,NaN,null,null,...,NaN,NaN,NaN,null,NaN,null,null,false,346,null


## Write To A CSV

YAH: we are expanding the work flow

In [27]:
files_df_expanded.to_csv(r'CIT_Newly_added_Catalog_0521.csv', na_rep='NULL', index=False)

### Review 

In [42]:
files_df_reduced_as_csv = pd.read_csv("CIT_Newly_added_Catalog_0521.csv")

In [43]:
files_df_reduced_as_csv.head()

,id,plan_name,plan_url,plan_resolution,planning_method,aquisition,easement,stewardship,plan_timeframe,agency_lead,...,water_quality,resource_species,community_resilience,ecosystem_resilience,gulf_economy,related_state,status,is_new,existing_planid,username
0,NaN,Habitat Management Plan - Baldwin County Meado...,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,THE MOBILE PENINSULA - CORRIDOR MASTER PLAN,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Management Plan for the - Audubon Bird Sanctuary,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Apalachee Region Comprehensive Economic Develo...,http://www.nado.org/wp-content/uploads/2014/08...,Regional,NaN,Yes,Yes,Yes,NaN,NaN,...,Yes,Yes,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN
4,NaN,Fishery Management Plan for Spanish Mackerel,http://sedarweb.org/docs/wsupp/S17RD03%20ASMFC...,GCR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Manage Spanish mackerel resourse,NaN,NaN,Minimize disruptions of markets for Spanish ma...,NaN,NaN,NaN,NaN,NaN


# Trouble Shooting

Fix any issues and blockers faced here with some experimentation.

## FXD BLQ 0: 'is_new' column with invalid Boolean values

FIX: Set values in Boolean columns to exactly True or False. Nan does not count.

**NOTE:** `True` in Python may not register as `TRUE` or `t` in SQL. It may not be projecting properly. Try this fix first.

In [ ]:
# files_ascsv = files_ascsv.assign(is_new="TRUE")

In [ ]:
# files = files_ascsv

In [ ]:
# files.to_csv(r'CIT_Newly_added_Catalog_0521.csv', index=False)

In [ ]:
# files.head()

In [ ]:
# files.to_csv(r'CIT_Newly_added_Catalog_0521.csv', na_rep='NULL', index=False)

## FXD BLQ 1: 
ERROR:  extra data after last expected column 
CONTEXT:  COPY plans, line 1: ",Unnamed: 0,Unnamed: 0.1,id,plan_name,date_added,suggested_by,url,plan_resolution,planning_method,aq..."

In [ ]:
# files.head()

## FXD BLQ 2: 

ERROR:  invalid input syntax for type integer: "id"
CONTEXT:  COPY plans, line 1, column id: "id"

In [ ]:
# files.head()

In [ ]:
# files['is_new']

## FXD BLQ 3: 

**SOL** make sure you use HEADER yes option.

ERROR:  invalid input syntax for type integer: "id"
CONTEXT:  COPY plans, line 1, column id: "id"

In [ ]:
# files.head()

## BLQ 4: Fixing Misaligned Columns From Data Process

It looks like some are just out of order.

In [10]:
# list(files.columns)

['Plan Name',
 'Date Added',
 'Suggested By',
 'Url',
 'Plan Resolution',
 'Planning Method',
 'Aquisition',
 'Easement ',
 'Stewardship',
 'Habitat',
 'Water Quality ',
 'Resources/Species',
 'Community Resilience',
 'Gulf Economy',
 'Code']

**NOTES** See below for plans SQL from PGAdmin

CREATE TABLE public.plans
(
    id integer NOT NULL DEFAULT nextval('plans_id_seq'::regclass),
    plan_name text COLLATE pg_catalog."default" NOT NULL,
    plan_url text COLLATE pg_catalog."default" NOT NULL,
    plan_resolution text COLLATE pg_catalog."default",
    planning_method text COLLATE pg_catalog."default",
    acquisition text COLLATE pg_catalog."default",
    easement text COLLATE pg_catalog."default",
    stewardship text COLLATE pg_catalog."default",
    plan_timeframe text COLLATE pg_catalog."default",
    agency_lead text COLLATE pg_catalog."default",
    geo_extent text COLLATE pg_catalog."default",
    habitat text COLLATE pg_catalog."default",
    water_quality text COLLATE pg_catalog."default",
    resources_species text COLLATE pg_catalog."default",
    community_resilience text COLLATE pg_catalog."default",
    ecosystem_resilience text COLLATE pg_catalog."default",
    gulf_economy text COLLATE pg_catalog."default",
    related_state text COLLATE pg_catalog."default",
    status text COLLATE pg_catalog."default",
    is_new boolean,
    existing_planid integer,
    username text COLLATE pg_catalog."default",
    CONSTRAINT plans_pkey PRIMARY KEY (id)